# Importações e Preparação do Dataset

In [1]:
# Importações necessárias
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB, CategoricalNB
from sklearn.linear_model import LinearRegression, Perceptron
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OrdinalEncoder

In [2]:
# Carregar o dataset
df = pd.read_csv("Covid Data.csv")

In [3]:
# Usei para não errar o nome das colunas no pré-processamento
print(df.columns)

Index(['USMER', 'MEDICAL_UNIT', 'SEX', 'PATIENT_TYPE', 'DATE_DIED', 'INTUBED',
       'PNEUMONIA', 'AGE', 'PREGNANT', 'DIABETES', 'COPD', 'ASTHMA', 'INMSUPR',
       'HIPERTENSION', 'OTHER_DISEASE', 'CARDIOVASCULAR', 'OBESITY',
       'RENAL_CHRONIC', 'TOBACCO', 'CLASIFFICATION_FINAL', 'ICU'],
      dtype='object')


# Pré-processamento

In [4]:
# Substituir valores específicos por NaN
df.replace({97: None, 99: None, '9999-99-99': None}, inplace=True)

# Verificar e imputar valores faltantes
imputer = SimpleImputer(strategy='most_frequent')
df_imputed = pd.DataFrame(imputer.fit_transform(df), columns=df.columns)

In [5]:
# Seleção de atributos e target
X = df_imputed.drop(columns=['DATE_DIED', 'CLASIFFICATION_FINAL'])
y = df_imputed['CLASIFFICATION_FINAL'].apply(lambda x: 1 if x < 4 else 0)  # 1: risco alto, 0: risco baixo

In [6]:
# Divisão em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [7]:
# Função para treinar e avaliar os modelos
def train_and_evaluate_model(model, X_train, y_train, X_test, y_test, model_name):
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    print(f"{model_name}")
    print(classification_report(y_test, y_pred))

In [ ]:
# Modelos que precisam de normalização
models_to_normalize = {
    "Support Vector Machine (padrão)": SVC(),
    "Support Vector Machine (kernel poli)": SVC(kernel='poly'),
    "KNN (padrão)": KNeighborsClassifier(),
    "KNN (k=5)": KNeighborsClassifier(n_neighbors=5),
    "KNN (métrica minkowski)": KNeighborsClassifier(metric='minkowski'),
    "Perceptron": Perceptron()
}

In [8]:
# Normalização
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

for model_name, model in models_to_normalize.items():
    train_and_evaluate_model(model, X_train_scaled, y_train, X_test_scaled, y_test, model_name)

In [ ]:
# Modelos que não precisam de normalização
models_no_normalize = {
    "Árvore de Decisão CART": DecisionTreeClassifier(),
    "Gaussian Naive Bayes (padrão)": GaussianNB(),
    "Linear Discriminant Analysis (LDA)": LinearDiscriminantAnalysis()
}

for model_name, model in models_no_normalize.items():
    train_and_evaluate_model(model, X_train, y_train, X_test, y_test, model_name)

In [ ]:
# Pré-processamento para Categorical Naive Bayes
categorical_features = X.select_dtypes(include=['object', 'category']).columns
numerical_features = X.select_dtypes(exclude=['object', 'category']).columns

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OrdinalEncoder())
])

X_cat = categorical_transformer.fit_transform(X[categorical_features])
y_cat = y.copy()  # Target remains the same

In [ ]:
# Divisão em treino e teste para os dados categóricos
X_train_cat, X_test_cat, y_train_cat, y_test_cat = train_test_split(X_cat, y_cat, test_size=0.2, random_state=42)

cat_nb = CategoricalNB()
train_and_evaluate_model(cat_nb, X_train_cat, y_train_cat, X_test_cat, y_test_cat, "Categorical Naive Bayes (padrão)")

In [9]:
# Verificar se há valores faltantes após a imputação
print("Verificação de valores faltantes:")
print("Valores faltantes em X_train:", pd.isnull(X_train).sum().sum())
print("Valores faltantes em X_test:", pd.isnull(X_test).sum().sum())

Verificação de valores faltantes:
Valores faltantes em X_train: 0
Valores faltantes em X_test: 0


In [10]:
# Função para treinar e avaliar os modelos
def train_and_evaluate_model(model, X_train, y_train, X_test, y_test, model_name):
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    print(f"{model_name}")
    print(classification_report(y_test, y_pred))

# Árvore de Decisão CART (padrão)

In [11]:
cart = DecisionTreeClassifier()
train_and_evaluate_model(cart, X_train, y_train, X_test, y_test, "Árvore de Decisão CART")


Árvore de Decisão CART
              precision    recall  f1-score   support

           0       0.67      0.89      0.76    131424
           1       0.58      0.25      0.35     78291

    accuracy                           0.65    209715
   macro avg       0.62      0.57      0.56    209715
weighted avg       0.64      0.65      0.61    209715



# Support Vector Machine (padrão)

In [ ]:
svm = SVC()
train_and_evaluate_model(svm, X_train, y_train, X_test, y_test, "Support Vector Machine (padrão)")

# Support Vector Machine (mudando o kernel por outro)

In [ ]:
svm_poly = SVC(kernel='poly')
train_and_evaluate_model(svm_poly, X_train, y_train, X_test, y_test, "Support Vector Machine (kernel poli)")

# KNN (padrão)

In [ ]:
knn = KNeighborsClassifier()
train_and_evaluate_model(knn, X_train, y_train, X_test, y_test, "KNN (padrão)")

# KNN (mudando o valor de k)

In [ ]:
knn_k5 = KNeighborsClassifier(n_neighbors=5)
train_and_evaluate_model(knn_k5, X_train, y_train, X_test, y_test, "KNN (k=5)")

# KNN (mudando a medida de distância)

In [ ]:
knn_minkowski = KNeighborsClassifier(metric='minkowski')
train_and_evaluate_model(knn_minkowski, X_train, y_train, X_test, y_test, "KNN (métrica minkowski)")

# Gaussian Naive Bayes (padrão)

In [ ]:
gnb = GaussianNB()
train_and_evaluate_model(gnb, X_train, y_train, X_test, y_test, "Gaussian Naive Bayes (padrão)")

# Categorical Naive Bayes (padrão), considerando todos os atributos categóricos

In [ ]:
# Pré-processamento para Categorical Naive Bayes
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OrdinalEncoder())
])

X_cat_imputed = categorical_transformer.fit_transform(X_cat)

# Divisão em treino e teste para os dados categóricos
X_train_cat, X_test_cat, y_train_cat, y_test_cat = train_test_split(X_cat_imputed, y, test_size=0.2, random_state=42)

cat_nb = CategoricalNB()
train_and_evaluate_model(cat_nb, X_train_cat, y_train_cat, X_test_cat, y_test_cat, "Categorical Naive Bayes (padrão)")

# 1.9 Regressão Linear (quadrados mínimos), regressão Linear não é adequada para classificação binária, então usamos LinearDiscriminantAnalysis

In [ ]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
lda = LinearDiscriminantAnalysis()
train_and_evaluate_model(lda, X_train, y_train, X_test, y_test, "Linear Discriminant Analysis (LDA)")

# Perceptron

In [ ]:
perceptron = Perceptron()
train_and_evaluate_model(perceptron, X_train, y_train, X_test, y_test, "Perceptron")